In [13]:
import pandas as pd
import numpy as np

import scipy.sparse

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from metrics import Evaluator
from time import perf_counter

In [2]:
mypath = "/home/mmarzec12/data/"
savepath = "/home/mmarzec12/models/jncf/"

explicit = pd.read_csv(mypath+"explicit_train.csv")
validation = pd.read_csv(mypath+"leave_one_out_validation.csv")


# list with (user,item) tuples from validation set
validation_list = [(u,i) for u,i in zip(validation.user_name, validation.game_id)]
# dict with user:game key-value pairs from validation set
validation_dict = {u:i for u,i in zip(validation.user_name, validation.game_id)}

# unique games and users
unique_users = explicit.user_name.unique()
unique_games = explicit.game_id.unique()

n_users, n_items = len(unique_users), len(unique_games)

# dictonaries to map users to unique ids and vice vers
us_to_ids = {u:i for i,u in enumerate(unique_users)}
ids_to_us = {i:u for i,u in enumerate(unique_users)}

# dictonaries to map games to unique ids and vice vers
gs_to_ids = {g:i for i,g in enumerate(unique_games)}
ids_to_gs = {i:g for i,g in enumerate(unique_games)}


implicit = pd.read_csv(mypath+"implicit_train.csv")

# filtering explicit ratings: filter ratings <6 and >=1
print(f"There is {np.sum(explicit.score <= 6)} rows with score <= 6.")
explicit = explicit[explicit.score > 6]

# we join implictit and explicit rating data
joined = pd.concat([explicit, implicit])
joined = joined[["user_name", "game_id", "score"]]
# converting all interaction data to "1" 
joined["score"] = 1

# creating sparse matrix with data
row = [us_to_ids[us] for us in joined.user_name]
col = [gs_to_ids[g] for g in joined.game_id]
data = joined.score

user_matrix = scipy.sparse.coo_matrix((data, (row, col)), shape=(len(unique_users), len(unique_games))).tocsr()
item_matrix = user_matrix.T.copy()
dok_matrix = user_matrix.todok()

user_loc = row
item_loc = col
ratings = data.values

There is 1362961 rows with score <= 6.


0

In [26]:
class JNCF(nn.Module):
    
    def __init__(self, n_users, n_items, DF_layers=[128, 64, 32], DI_layers=[64, 8], combination="concatenation"):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.combination = combination
        self.embed_dim = DF_layers[-1]
        
        if self.combination == 'concatenation':
            self.embed_dim *= 2
        elif self.combination == 'multiplication':
            pass
        else:
            raise ValueError('combination type should be "concatenation" or "multiplication" !')
        
        self.DI_layers = self.initialize_layers(self.embed_dim, DI_layers)
        self.DF_users = self.initialize_layers(n_items, DF_layers)
        self.DF_items = self.initialize_layers(n_users, DF_layers)
        self.prediction_layer = nn.Linear(DI_layers[-1], 1)
        
        
    def initialize_layers(self, n_initial, layers, nonlinearity="relu"):
        res = []
        for i in range(len(layers)):
            if i == 0:
                layer = nn.Linear(n_initial, layers[i])
            else:
                layer = nn.Linear(layers[i-1], layers[i])
                
            nn.init.normal_(layer.weight, 0, 0.01)
            layer.bias.data.normal_(0.0, 0.01)
            res.append(layer)
            res.append(nn.ReLU())
        
        #print(res)
        return nn.Sequential(*res)
    
    
    def forward(self, user, item_i, item_j):
        zu = self.DF_users(user)
        zi = self.DF_items(item_i)
        zj = self.DF_items(item_j)
        
        if self.combination == "concatenation":
            i_feature_vector = torch.cat((zu, zi), dim=-1)
            j_feature_vector = torch.cat((zu, zj), dim=-1)
        elif self.combination == "multiplication":
            i_feature_vector = zu * zi
            j_feature_vector = zu * zj
        
        y_i = self.prediction_layer(self.DI_layers(i_feature_vector))
        y_j = self.prediction_layer(self.DI_layers(j_feature_vector))
        return y_i.view(-1), y_j.view(-1)
    
# utlis function
def TOP1(item_i, item_j, n_negs):
    diff = item_j - item_i
    loss = (torch.sigmoid(diff) + torch.sigmoid(torch.pow(item_j, 2)))
    return torch.mean(loss)

In [29]:
user_idxlist, item_idxlist = list(range(n_users)), list(range(n_items))

# list with indices, len=numb_of_obs
idxlist = np.array(range(len(user_loc)))

# model
model = JNCF(n_users, n_items)

lr = 1e-3
n_epochs = 10
batch_size = 128
n_negs = 10
alpha = 0.8

# validation
sample_size_val = 100
k = 10
NDCGs = []
ERRs = []
HRs = []

point_loss_function = nn.NLLLoss() # nn.MSELoss()
pair_loss_function = TOP1
opt = torch.optim.Adam(model.parameters(), lr=lr)

In [33]:
for epoch in range(n_epochs):
    
    np.random.shuffle(idxlist)
    epoch_loss, epoch_pair_loss, epoch_point_loss = .0, .0, .0
    
    # Training phase
    
    print("Training phase...")
    start = perf_counter()
    for batch_idx, start_idx in enumerate(tqdm(range(0, len(idxlist), batch_size))):
        end_idx = min(len(idxlist), start_idx+batch_size)
        idx = idxlist[start_idx:end_idx]
        
        # u_ids - list of user ids, but taken from the list of user ids from
        # all interactions, so there are multiple instances of specific user id.
        # This means that users with higher number of interactions are more likely
        # to be picked here.
        u_ids = user_loc[start_idx:end_idx]
        i_ids = item_loc[start_idx:end_idx]
        rs = ratings[start_idx:end_idx]
        
        # We select input for the network. In case of users we pick all
        # their interaction history, in case of items all their interaction 
        # history (item input has length=n_users)
        users = torch.FloatTensor(user_matrix[u_ids,].toarray())
        #print(users.shape)
        items = torch.FloatTensor(item_matrix[i_ids,].toarray())
        labels = torch.LongTensor(rs)
        
    
         # Negative Sampling
        neg_items_list = []
        to_sample = list(range(n_items))
        for _ in range(0, n_negs):
            neg_res = []
            for u in u_ids:
                cond = 0 
                while cond == 0:
                    negid = np.random.choice(to_sample)
                    if dok_matrix[u, negid] != 0:
                        neg_res.append(negid)
                        cond = 1
            neg_items_list.append(neg_res)

        for neg_idx in range(0, n_negs):
            # we start learning procedure
            
            opt.zero_grad()
            point_loss, pair_loss = 0., 0.

            neg_ids = neg_items_list[neg_idx]
            items_j = torch.FloatTensor(item_matrix[neg_ids].toarray())

            y_i, y_j = model.forward(users, items, items_j)

            point_loss = point_loss_function(y_i, labels)  # positive items i
            pair_loss = pair_loss_function(y_i, y_j, n_negs)

            loss = alpha * pair_loss + (1 - alpha) * point_loss

            epoch_loss += loss.item()
            epoch_pair_loss += pair_loss.item()
            epoch_point_loss += point_loss.item()

            loss.backward()
            opt.step()
            
    train_time = perf_counter() - start_time
        
    
    # Evaluation phase
    print("Evaluation phase...")
    model.eval()
    val_res = {}
    to_sample = list(ids_to_gs.keys())
    for user in tqdm(unique_users):
        #print(user)
        selected = np.random.choice(to_sample, sample_size_val, replace=False)
        selected  = np.append(selected, gs_to_ids[validation_dict[user]])
        
        
        usr_id = [us_to_ids[user]] * len(selected)
        usr = torch.FloatTensor(user_matrix[usr_id].toarray())
        
        itms = torch.FloatTensor(item_matrix[selected].toarray())
        #print(usr.shape, itms.shape)
        
        preds = model.forward(usr, itms)
        #print(preds.shape)
        _, rec_ids = torch.topk(preds, k)
        rec_ids = [ids_to_gs[selected[i]] for i in rec_ids]
        val_res[user] = rec_ids
    
    evaluator = Evaluator(k=k, true=validation_list, predicted=val_res)
    evaluator.calculate_metrics()
    ndcg10, err10, hr10 = ev.ndcg, ev.err, ev.hr
    
    NDCGs.append(ndcg10)
    ERRs.append(err10)
    HRs.append(hr10)

Training phase...


  0%|          | 0/73544 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [76]:
user_matrix = scipy.sparse.coo_matrix((data, (row, col)), shape=(len(unique_users), len(unique_games))).tocsr().toarray()
item_matrix = user_matrix.T.copy()

4

In [14]:
torch.set_num_threads(2)

In [22]:
for epoch in range(n_epochs):
    
    epoch_loss = 0
    np.random.shuffle(idxlist)
    
    #start = perf_counter()
    # Training phase
    """
    print("Training phase...")
    for batch_idx, start_idx in enumerate(tqdm(range(0, len(idxlist), batch_size))):
        end_idx = min(len(idxlist), start_idx+batch_size)
        idx = idxlist[start_idx:end_idx]
        
        # u_ids - list of user ids, but taken from the list of user ids from
        # all interactions, so there are multiple instances of specific user id.
        # This means that users with higher number of interactions are more likely
        # to be picked here.
        u_ids = user_loc[start_idx:end_idx]
        i_ids = item_loc[start_idx:end_idx]
        rs = ratings[start_idx:end_idx]
        
        # We select input for the network. In case of users we pick all
        # their interaction history, in case of items all their interaction 
        # history (item input has length=n_users)
        users = torch.FloatTensor(user_matrix[u_ids,])
        #print(users.shape)
        items = torch.FloatTensor(item_matrix[i_ids,])
        labels = torch.LongTensor(rs)
        
        # todo: implement pair-wise learning
        
        opt.zero_grad()
        y_hat = model.forward(users, items)
        #print(y_hat)
        loss = pointwise_loss(y_hat, labels) 
        #print(batch_idx)
        #print(f"batch loss = {loss.item()}")
        epoch_loss += loss.item()
        
        loss.backward()
        opt.step()
    """
        
    
    # Evaluation phase
    print("Evaluation phase...")
    model.eval()
    val_res = {}
    to_sample = list(ids_to_gs.keys())
    for user in tqdm(unique_users):
        #print(user)
        selected = np.random.choice(to_sample, sample_size_val, replace=False)
        selected  = np.append(selected, gs_to_ids[validation_dict[user]])
        
        
        usr_id = [us_to_ids[user]] * len(selected)
        usr = torch.FloatTensor(user_matrix[usr_id])
        
        itms = torch.FloatTensor(item_matrix[selected])
        #print(usr.shape, itms.shape)
        
        preds = model.forward(usr, itms)
        #print(preds.shape)
        _, rec_ids = torch.topk(preds, k)
        rec_ids = [ids_to_gs[selected[i]] for i in rec_ids]
        val_res[user] = rec_ids
    
    evaluator = Evaluator(k=k, true=validation_list, predicted=val_res)
    evaluator.calculate_metrics()
    ndcg10, err10, hr10 = ev.ndcg, ev.err, ev.hr
    
    NDCGs.append(ndcg10)
    ERRs.append(err10)
    HRs.append(hr10)    

Evaluation phase...


  0%|          | 0/109084 [00:00<?, ?it/s]

KeyboardInterrupt: 